In [ ]:
import cv2
import numpy as np
import os
import tensorflow as tf

# Assuming these constants are defined elsewhere
IMG_HEIGHT = 128
IMG_WIDTH = 128
CLASS_LABELS = ['Abuse','Arrest','Arson','Assault','Burglary','Explosion','Fighting','RoadAccidents','Robbery','Shooting','Shoplifting','Snatching','Stealing','Vandalism']
preprocess_fun = tf.keras.applications.densenet.preprocess_input
model = tf.keras.models.load_model('F:\\Model_Saved\\Trained\\ADmodel.keras')  # Load your trained model

def extract_frames(video_path, output_dir, frame_rate=10):
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    step = int(fps / frame_rate)
    
    os.makedirs(output_dir, exist_ok=True)
    
    count = 0
    frames_paths = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if count % step == 0:
            frame_id = int(count / step)
            frame_path = os.path.join(output_dir, f"frame_{frame_id:04d}.jpg")
            cv2.imwrite(frame_path, frame)
            frames_paths.append(frame_path)
        count += 1
    cap.release()
    
    return frames_paths

def preprocess_frame(frame_path, img_height, img_width, preprocess_fun):
    img = tf.keras.preprocessing.image.load_img(frame_path, target_size=(img_height, img_width))
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = preprocess_fun(img)
    img = np.expand_dims(img, axis=0)
    return img

def display_final_counts(activity_counts):
    final_frame = np.zeros((600, 800, 3), dtype=np.uint8)
    
    # Display all detected activities and their counts
    y_offset = 50
    for label, count in activity_counts.items():
        text = f'{label}: {count}'
        cv2.putText(final_frame, text, (50, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        y_offset += 30
    
    cv2.namedWindow('Final Activity Counts', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Final Activity Counts', 800, 600)
    cv2.imshow('Final Activity Counts', final_frame)
    
    # Keep the window open until the user manually closes it
    while True:
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cv2.destroyAllWindows()

def display_video_with_activity(video_path, output_dir, confidence_threshold):
    frames_paths = extract_frames(video_path, output_dir)
    
    # Dictionary to store detected activity counts
    activity_counts = {label: 0 for label in CLASS_LABELS}
    
    cv2.namedWindow('Activity Detection', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Activity Detection', 800, 600)
    
    for frame_path in frames_paths:
        frame = cv2.imread(frame_path)
        
        # Get the current window size
        height, width = cv2.getWindowImageRect('Activity Detection')[3], cv2.getWindowImageRect('Activity Detection')[2]
        
        # Resize the frame to fit the window
        frame_resized = cv2.resize(frame, (width, height))
        
        # Preprocess the frame
        preprocessed_frame = preprocess_frame(frame_path, IMG_HEIGHT, IMG_WIDTH, preprocess_fun)
        
        # Predict using the model
        predictions = model.predict(preprocessed_frame)
        predicted_class = np.argmax(predictions)
        confidence = predictions[0][predicted_class]
        
        # Determine the activity label
        activity_label = 'Normal'
        if confidence >= confidence_threshold:
            activity_label = CLASS_LABELS[predicted_class]
            activity_counts[activity_label] += 1
        
        # Display the frame with activity label
        cv2.putText(frame_resized, f'Activity: {activity_label}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow('Activity Detection', frame_resized)
        
        # Press 'q' to quit the video
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cv2.destroyAllWindows()
    
    # Display final counts
    display_final_counts(activity_counts)

#video_path = "C:\\Users\\lalit\\dataset\\Anomaly_Videos\\Robbery\\Robbery008_x264.mp4"
video_path = "C:\\Users\\lalit\\Desktop\\New folder\\New folder\\gun.mp4"
output_dir = "output_frames"

display_video_with_activity(video_path, output_dir, confidence_threshold=0.2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━